<a href="https://colab.research.google.com/github/IoanPJ/Final_year_project_IMPJ/blob/main/ML/imbalanced_CL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import imblearn
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, log_loss, roc_auc_score, RocCurveDisplay, roc_curve
import numpy as np
import pandas as pd
#from IMPJ import DataProcessor
import random
import matplotlib.pyplot as plt
from os.path import basename, exists
import matplotlib as mpl
import random
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsRestClassifier

#dp = DataProcessor()

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/AstronomicalData/raw/main/' + 'az-paper-twocol.mplstyle')
plt.style.use('./az-paper-twocol.mplstyle')

Downloaded az-paper-twocol.mplstyle


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
test_size = 0.3

filepath = "/content/drive/My Drive/Colab Notebooks/chiaro_12.csv"
data = pd.read_csv(filepath, index_col=0)
data = data.dropna()
Y = data['CLASS1']
X = data.drop(labels='CLASS1',axis=1)
#X,Y = dp.choose_2_vars('CLASS1',data,1,2)

In [10]:
sm = SMOTE(random_state=42)

Y[Y==2]=1 # non radio galaxies
Y[Y==3]=0 # radio galaxies

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size,random_state=2)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print(f'Original Dataset Shape: {Counter(y_train)}')
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)
print(f'Resampled dataset shape {Counter(y_train_res)}')


Original Dataset Shape: Counter({1: 1618, 0: 34})
Resampled dataset shape Counter({1: 1618, 0: 1618})


<ipython-input-10-5ac4a2343a20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[Y==2]=1 # non radio galaxies
<ipython-input-10-5ac4a2343a20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[Y==3]=0 # radio galaxies


In [ ]:
NN = MLPClassifier(activation='tanh',hidden_layer_sizes=(50,50),learning_rate='constant',solver='sgd',
                   max_iter=5000,random_state=5)
BNN = BaggingClassifier(estimator=NN,n_estimators=20,bootstrap=True,verbose=50,n_jobs=-1)
#ovr = OneVsRestClassifier(estimator=BNN, n_jobs=-1,verbose=10)
BNN.fit(x_train_res,y_train_res)
y_pred = BNN.predict(x_test)
y_proba = BNN.predict_proba(x_test)


Building estimator 1 of 20 for this parallel run (total 20)...
Building estimator 2 of 20 for this parallel run (total 20)...
Building estimator 3 of 20 for this parallel run (total 20)...
Building estimator 4 of 20 for this parallel run (total 20)...
Building estimator 5 of 20 for this parallel run (total 20)...
Building estimator 6 of 20 for this parallel run (total 20)...
Building estimator 7 of 20 for this parallel run (total 20)...
Building estimator 8 of 20 for this parallel run (total 20)...
Building estimator 9 of 20 for this parallel run (total 20)...
Building estimator 10 of 20 for this parallel run (total 20)...


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Building estimator 11 of 20 for this parallel run (total 20)...
Building estimator 12 of 20 for this parallel run (total 20)...


In [ ]:
print(np.unique(y_train))

''' SCORING METRICS '''

accuracy = accuracy_score(y_test, y_pred)*100
confusion = confusion_matrix(y_test, y_pred)
#roc_auc = roc_auc_score(y_test,y_pred)
#roc_auc_weightedavg = roc_auc_score(y_test,y_pred,average='weighted')
logloss = log_loss(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
fpr, tpr, thresholds = roc_curve(y_test,y_proba.T[1])

print('The Neural Network accuracy is ' + str(accuracy))
#print('The Neural Network ROC AUC Scores are: '+str(roc_auc))
#print("The Neural Network's Weighted Average ROC AUC Score is: " + str(roc_auc_weightedavg))
print("The Neural Network's Logarithmic Loss Score is: " + str(logloss))
print('The Neural Network F1 Score is: '+str(f1))
print('The Neural Network Confusion Matrix is:')
print(confusion)

[1 2 3]


ValueError: y_true and y_pred contain different number of classes 3, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [1 2 3]